In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from skimage.feature import hog
import skimage.io as io
from skimage.util import view_as_blocks
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
import pickle
import numpy as np
import warnings
import math

In [2]:
# Fit encoders in order to map string labels to integers
# The fact that we are storing the scikit objects makes the encoding reproducible and consistent
# We do not need to fit each time, we store the fitted object and use its transform method when needed

label_encoder_specific = LabelEncoder().fit(pd.read_csv('dataset_cards/cards.csv')['labels'])  # Specific
label_encoder_category = LabelEncoder().fit(pd.read_csv('dataset_cards/cards.csv')['card type'])  # card type

### HOG

In [3]:
from skimage.feature import hog
from skimage import color

In [4]:
class CardsDataset(Dataset):
    """
    A PyTorch Dataset class specifically aimed at digesting and retrieving the images of the cards, together with their
    labels, which are also reliably encoded.
    """

    def __init__(self, csv_path: str, root_dir: str, transform: str,
                 card_category: bool, label_encoder: LabelEncoder,
                 subset: str = 'train') -> None:
        """
        Initialization of the CardDataset class
        :param csv_path: The path of the csv where to retrieve the necessary information about the images of the cards.
        :param root_dir: The root directory for the dataset.
        :param transform: Option between Histogram of Gradients ('hog') or Histogram of colors ('rgb_hist').
        Pass None in order to avoid transformations (normalization at pixel level is the only operation performed).
        :param card_category: Whether the label refers to the card category or to the specific card itself.
        :param label_encoder: The scikit-learn LabelEncoder object necessary to map the string labels to integers.
        :param subset: Choose between train ('train'), test ('test') and validation ('valid') dataset.
        """
        total_cards = pd.read_csv(csv_path)
        try:
            self.cards_table = total_cards[total_cards['data set'] == subset]
        except KeyError:
            raise KeyError('There is no subset in the dataset with the queried label')
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            self.cards_table['labels' if not card_category else 'card type'] = \
                label_encoder.transform(self.cards_table['labels']) if not card_category else \
                label_encoder.transform(self.cards_table['card type'])
        self.root_dir = root_dir
        self.transform = transform
        self.card_category = card_category

    def __len__(self):
        return len(self.cards_table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        card_class = self.cards_table.iat[idx, 2] if not self.card_category else self.cards_table.iat[idx, 3]

        if self.transform == 'hog':
            representation = np.load(os.path.join(self.root_dir, 'hog',
                                                  self.cards_table.iat[idx, 1].split('.')[0] + '.npy'))
            representation = np.moveaxis(representation, 2, 0)
            
        elif self.transform == 'hog_2':
            img_path = os.path.join(self.root_dir, self.cards_table.iat[idx, 1])
            image = color.rgb2gray(io.imread(img_path))
            fd, hog_image = hog(image, orientations=6, pixels_per_cell=(22, 22),
                    cells_per_block=(1, 1), block_norm="L1", visualize=True, feature_vector = True)
            representation = fd
            
        elif self.transform is None or self.transform == 'rgb_hist':
            img_path = os.path.join(self.root_dir, self.cards_table.iat[idx, 1])
            image = io.imread(img_path)
            if self.transform == 'rgb_hist':
                representation, _ = np.histogramdd((np.reshape(image, (-1, 3))), bins=[np.linspace(0, 256, 6)] * 3)
                representation = (representation / representation.sum()).flatten()
            else:
                representation = np.moveaxis(image/255.0, 2, 0)

        else:
            raise ValueError(f'There is no transformation function for \'{self.transform}\'')

        return representation.astype(np.float32), card_class

### Training set

In [5]:
training_hog = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                            transform='hog_2', card_category=False,
                            label_encoder=label_encoder_specific, subset='train')

In [13]:
# X = np.array([i[0] for i in training_hog])
# y = np.array([i[1] for i in training_hog])
# np.save('hog.npy', X)
# np.save('hogy.npy', y)

In [6]:
X = np.load('hog.npy')
y = np.load('hogy.npy')

In [19]:
print(X.shape)
print(y.shape)

(7624, 600)
(7624,)


### Test set

In [8]:
test_hog = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                        transform='hog_2', card_category=False,
                        label_encoder=label_encoder_specific, subset='test')

In [9]:
X_t = np.array([i[0] for i in test_hog])
y_t = np.array([i[1] for i in test_hog])

In [10]:
print(X_t.shape)
print(y_t.shape)

(265, 600)

### model

In [27]:
clf = LogisticRegression(multi_class="multinomial", n_jobs=os.cpu_count()//2, solver="sag", verbose=1).fit(X, y)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 91 seconds


C:\Users\iadig\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  1.5min finished


In [28]:
print('Accuracy:', str(round(clf.score(X_t, y_t), 4)*100)+"%")

Accuracy: 65.66%


In [23]:
y_pred = clf.predict(X_t)
print((y_pred == y_t).sum())
print(y_pred == y_t)

174
[False False False  True False False  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True False  True False  True False  True  True  True  True
 False False False  True  True False False False False  True  True  True
  True False  True False  True  True  True False False False False  True
 False False  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True False False  True  True False  True  True
  True  True  True False  True False  True False  True  True  True  True
  True False  True  True False False False False False  True  True  True
  True False  True  True  True False  True False False  True  True  True
 False False  True  True False False  True  True False False  True  True
 False  True  True  True  True False False  True  True False  True  True
  True False  True  True False  True  True  True False  True  True  True
  True False False  True  True  True  True  Tru

In [33]:
# from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [48]:
# model = HistGradientBoostingClassifier(max_bins=224, max_iter=100).fit(X[0][0], y)
# define the evaluation procedure
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# evaluate the model and collect the scores

In [50]:
# n_scores = cross_val_score(model, X_t, y_t, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))